# Data Processing

In [ ]:
### rename
my_data = my_data.rename(columns = {'x1':'pred_1', 'x2':'pred_2'})

### to see if data is balanced
rice.CLASS.value_counts()

### converting string to integer
y = y.astype(np.uint8)

### data types
df.dtypes

### null/NaN values
df.isnull().any()

### for both null and data types
df.info()

### fill null values
df= df.fillna('M')

### stats summary
df.describe()

### one hot encoding
df = pd.get_dummies(df,columns=["Sex", "BP","Cholesterol"])

### Setting binary values
df.loc[-df["Drug"].isin(['DrugY']),"Drug"]="0"
df.loc[df["Drug"].isin(['DrugY']),"Drug"]="1"

### splitting for test and train
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.25,random_state=11)

### correlation - note that c is of type pandas dataframe
c = df3.corr()

### for sorting
df.sort_values(ascending=True)


# Metrics

In [ ]:
### baseline accuracy
A  = rice.CLASS[rice['CLASS']=='Osmancik'].count() # majority class
B = rice.CLASS[rice['CLASS']=='Cammeo'].count()

baselineacc = A/(A+B) # the classifier you train must beat this number else no point in training one

print('Baseline Accuracy: '+str((baselineacc*100).round(1))+'%')

### label-base accuracy, precision, specificty and recall
def compute_performance(yhat, y, classes):
    # First, get tp, tn, fp, fn
    tp = sum(np.logical_and(yhat == classes[1], y == classes[1]))
    tn = sum(np.logical_and(yhat == classes[0], y == classes[0]))
    fp = sum(np.logical_and(yhat == classes[1], y == classes[0]))
    fn = sum(np.logical_and(yhat == classes[0], y == classes[1]))

    print(f"tp: {tp} tn: {tn} fp: {fp} fn: {fn}")
    
    # Accuracy
    acc = (tp + tn) / (tp + tn + fp + fn)
    
    # Precision
    # "Of the ones I labeled +, how many are actually +?"
    precision = tp / (tp + fp)
    
    # Recall
    # "Of all the + in the data, how many do I correctly label?"
    recall = tp / (tp + fn)    
    
    # Sensitivity
    # "Of all the + in the data, how many do I correctly label?"
    sensitivity = recall
    
    # Specificity
    # "Of all the - in the data, how many do I correctly label?"
    specificity = tn / (fp + tn)
    
    # Print results
    
    print("Accuracy:",round(acc,3),"Recall:",round(recall,3),"Precision:",round(precision,3),
          "Sensitivity:",round(sensitivity,3),"Specificity:",round(specificity,3))

compute_performance(ytest_hat, ytest, ricelr.classes_)

### ROC ranking-based metric for classification
fpr, tpr, _ = roc_curve(ytest, ytest_prob[:,1], pos_label=ricelr.classes_[1]) # 2nd arg: ranking score, 3rd arg: "Osmancik"
ax =sns.lineplot(x=fpr,y=tpr)
ax.set_xlabel("FP Rate")
ax.set_ylabel("TP Rate")

### auc
auc(fpr,tpr)

### confusion matrix for classification
confusion_matrix(yhat,y).T
# We transpose to get it in the format we saw in slides:
# Rows: Predicted labels
# Columns: True labels 

### accuracy via cross validation score
scores = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")
print("Scores:", scores.round(2))
print("Mean:", scores.mean().round(2))
print("Standard deviation:", scores.std().round(2))

### for multi class
classification_report(y_test, clf4.predict(X_test)

### MSE/RMSE
mean_squared_error(ytrain, model.predict(Xtrain), squared=False)


# Plotting

In [ ]:
### matrix plots
pd.plotting.scatter_matrix(my_data, alpha = 0.2, diagonal='kde')

### joinplot - to see any variables need transformation
ax = sns.jointplot(x=df3.Overall, y=np.log(df3.Value), kind='scatter')
ax.ax_joint.set_xlabel('Overall')
ax.ax_joint.set_ylabel('log(Value)')
plt.show()
# # You can also plot the joint probability distribution
# kdeplot = sns.jointplot(data=D, x='weeks', y='weight', kind='kde', fill=True, cbar=True)
# plt.subplots_adjust(left=0.1, right=0.8, top=0.9, bottom=0.1)
# # get the current positions of the joint ax and the ax for the marginal x
# pos_joint_ax = kdeplot.ax_joint.get_position()
# pos_marg_x_ax = kdeplot.ax_marg_x.get_position()
# # reposition the joint ax so it has the same width as the marginal x ax
# kdeplot.ax_joint.set_position([pos_joint_ax.x0, pos_joint_ax.y0, pos_marg_x_ax.width, pos_joint_ax.height])
# # reposition the colorbar using new x positions and y positions of the joint ax
# kdeplot.fig.axes[-1].set_position([.83, pos_joint_ax.y0, .07, pos_joint_ax.height])

### numerical features all in one
df.hist(bins=15, figsize=(10,10))
plt.show()

# Week 1

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import scipy.stats as ss 
import scipy.optimize as so
from sklearn import linear_model

In [ ]:
### models
LinearRegression()

def linearModelPredict(b,X):
    yp = np.dot(X,b) #Xrow: obs Xcol: matches length of b. #brow: matches Xcol features. bcol: length of labels

    return yp

#Note as long as X's cols match b (a 1D array) length we're good. b could be a row or a column vector.
# the resulting yp will have shape of b.

def linearModelLossRSS(b,X,y):
    # The loss is really a function of b.  The b changes, the X and y stay fixed.
    # Make predictions
    predY = linearModelPredict(b,X)

    # Compute residuals.  This is an array.  The dimension of res will depend on if
    # b is 1d or 2d.  If b is 2d, predY will be 2d, and so res will be 2d due to something
    # called "array broadcasting".
    
    res = y-predY
    # Simply sum up the squared residuals.  This is the value of our loss.
    residual_sum_of_squares = sum(res**2) 
    
    # Because res is a vector, we can take the product of res with X.
    # Since X is two dimensional because it is a design matrix, this results in a
    # 2d array.  The gradient has three elements because there are three parameters.
    gradient=-2*np.dot(res,X)

    return (residual_sum_of_squares, gradient)

def linearModelLossLAD(b,X,y):
    # Same concept as before, different loss
    predY = linearModelPredict(b,X)
    res = y-predY
    sres = np.sign(res); 
    sum_abs_dev = sum(abs(res))
    # Note the gradients are computed using the sign of the residuals
    grad =- (np.dot(sres,X))

    return (sum_abs_dev,grad)


def linearModelFit(X,y,lossfcn = linearModelLossRSS):
    # Because we know b has to have the some dimension as X has columns,
    # We can use the number of columns to determine the size of betas
    # In this case, we use a 2d array
    nrows,ncols = X.shape
    betas=np.zeros((ncols,1))
    # Optimize the loss
    RES = so.minimize(lossfcn,betas,args=(X,y),jac=True, options={'disp': True})
    # Obtain estimates from the optimizer
    estimated_betas=RES.x 
    # Compute goodness of fit.
    diff = y-np.mean(y)
    TSS = sum(diff**2)
    RSS,deriv = linearModelLossRSS(estimated_betas,X,y)
    R2 = 1-RSS/TSS 

    return (estimated_betas,R2)

### fitting and plotting
    # Fit the data 
y = my_data.y.values
pred_1 = my_data.pred_1.values
N = pred_1.size
print(N)
X = np.c_[np.ones(N), pred_1]
betas, R2 = linearModelFit(X,y)

# Create new data
pred_grid = np.linspace(pred_1.min(), pred_1.max(), 100)
# Turn it into a design matrix
Xn = np.c_[np.ones(pred_grid.size), pred_grid]

# Compute predictions with the new data and estimated coefficients
yn = linearModelPredict(betas, Xn)

fig, ax = plt.subplots(dpi = 120)
my_data.plot.scatter(x='pred_1', y='y', alpha=0.75, ax=ax)
ax.set_xlabel('pred_1')
ax.set_ylabel('target')

ax.plot(pred_grid, yn, color = 'red')
ax.annotate('R\u00b2: {R2}'.format(R2=round(R2, 2)), 
            xy=(0.2, 0.8), 
            xycoords='axes fraction',
            ha='center',
            fontsize = 16)

ax.set_title('RSS Model')

plt.show()

# Week 2

In [ ]:
# Packages for this assignment
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# from scipy.optimize import minimize
import scipy.optimize as so
import seaborn as sns
# from sklearn.linear_model import LinearRegression

In [ ]:
def exponentialNegLogLikelihood(lamb, y): # specific to the problem
    likelihood = -np.sum(np.log(lamb)-lamb*y)
    return likelihood

def exponentialRegressionNegLogLikelihood(b, X, y):
    # Here we have to be carefull since the mean of the exponential distribution is not the same $\lambda$ parameter.
    lamb = np.exp(-X @ b)
    # Use exponentialNegLogLikelihood to compute the likelihood
    neg_log_lik = exponentialNegLogLikelihood(lamb, y)
    return neg_log_lik

def Prediction(b, X):
    yhat = np.exp(X @ b)
    return yhat

def Model_fit(X, y):
    # Instantiate a guess for the betas, beta_start, so that the optimizer has somewhere to start
    # Keep in mind what shape the beta_start should be. It shoud have the same number of elements as X as columns
    nrows, ncols = X.shape
    beta_start = np.zeros((ncols, 1))
    # Minimize the appropriate likelihood function
    mle = minimize(exponentialRegressionNegLogLikelihood, beta_start, args = (X, y), method = "Powell")
    # Extract the maximum likelihood estimates from the optimizer.
    betas = mle.x
    return betas

# Week 3

In [ ]:
### experimenting with threshold
threshold = 0.1
ytest_prob = ricelr.predict_proba(Xtest)
yhat = ricelr.classes_[(ytest_prob[:,1] > threshold).astype(int)]

### SGDClassifier
# This classifier has the advantage of being capable of handling very large datasets efficiently.
# Use for multiclass

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=seed) 
# AND
clf4 = make_pipeline(StandardScaler(), SGDClassifier(loss='log', penalty="l2", max_iter=2000, tol=1e-3, n_jobs=-1, random_state=seed)).fit(X_train, y_train)


### manual sigmoid probabilities
sigmoid = lambda x: 1 / (1 + np.exp(-x))
z1 = np.dot(Xtest[:,[0,1]],LOGREG_first_two.coef_.T) + LOGREG_first_two.intercept_
sigmoid(z1)[

# Week 4

In [ ]:
### bootstrap for means
def createBootstrapMeans(data, numboot=10000):
    n = len(data)
    boot_means = np.zeros(numboot)
    np.random.seed(0)
    for i in range(numboot):
        d = data.sample(n, replace=True)
        boot_means[i] = d.mean()
    return boot_means

### boostrap for coeffs
def BootstrapCoef(data, numboot=1000):
    n = len(data)
    theta = np.zeros((numboot,3))    
    for i in range(numboot):
        d = data.sample(n, replace=True)
        X_fit = np.c_[d.weeks,d.weeks**2,d.weeks**3]
        regr.fit(X_fit,d.weight)
        theta[i,:]=regr.coef_
    return theta

### bootstrap predictions
def BootstrapPred(data, xp, numboot=1000):
    n = len(data)
    X_pred = np.c_[xp,xp**2,xp**3]
    y_pred = np.zeros((numboot,X_pred.shape[0]))
    for i in range(numboot):
        d = data.sample(n, replace=True)
        X_fit = np.c_[d.weeks,d.weeks**2,d.weeks**3]
        regr.fit(X_fit,d.weight)
        y_pred[i,:]=regr.predict(X_pred)
    return y_pred

### bootsrap mean predictions
def BootstrapPred(MODEL, X_train, y_train, X_test, numboot=100):
    y_pred = np.zeros(numboot)
    for i in range(numboot):
        X_fit = X_train.sample(Xtrain.shape[0], replace=True)
        y_fit = y_train[X_fit.index]
        y_pred[i]=np.exp(MODEL.fit(X_fit, y_fit).predict(X_test)).mean() # bc we are working with log(y)
    return y_pred

### CLT
## Using central limit theorem, compute confidence interval

stderr = np.std(data.Match) / np.sqrt(len(data.Match))
print("Stderr: %.3f" % stderr)

# Area under a standard normal from -1.96 to 1.96 is about 95% (3 for 99.9% probability)
critval = 1.96 # critical value

# Confidence interval
norm_ci = [(data.Match.mean() - critval*stderr).round(3), 
           (data.Match.mean() + critval*stderr).round(3)]

print("Norm ci:",norm_ci)

# Stderr: 0.055
# Norm ci: [0.7, 0.916]

### t-distribution
# Get the critical values for t at an alpha = 0.05/2 (i.e., 5% divided by 2), and 52-1 = 51 dof.
alpha = (5/100)
crit_val = 1-(alpha/2)

# Degrees of freedom (dof) of an estimate is the number of independent
# pieces of information that went into calculating the estimate.
dof = n-1
# the precise shape of the t distribution depends on dof, which is related to the sample size.

# ppf: percent point function (inverse of cdf — percentiles).
# It returns the value x of the variable that has a given cumulative distribution probability (cdf).
# Thus, given the cdf(x) of a x value, ppf returns the value x itself, therefore, operating as the inverse of cdf.
t_value = t.ppf(crit_val, df=dof) # 1st arg: critical value; 2nd arg: dof

stderr = np.std(data.Match) / np.sqrt(len(data.Match))

t_ci = data.Match.mean() + t_value * stderr * np.array([-1, 1])
print('The t-based confidence interval is equal to {}'.format(t_ci.round(3)))

### boostrap CI
sns.set_style("darkgrid") 

# Create a dataframe
bm = pd.DataFrame(data=createBootstrapMeans(data)-data.Match.mean(), columns=['samples'])
# Bootstrap with CL of 95%
boot_CL = 95/100 
p_1 = (1-boot_CL)/2
p_2 = 1-p_1
boot_quant = np.quantile(bm, [p_1, p_2])
print('boot_quant:',boot_quant)

# Compute confidence interval
boot_ci = [(data.Match.mean() - boot_quant[1]).round(3), 
           (data.Match.mean() - boot_quant[0]).round(3)]
print("Boot Confidence Interval:",boot_ci)
boot_ci2 = [(data.Match.mean() - np.abs(boot_quant[0])).round(3), # absolute values bc numbers are negative since data is centered. 
           (data.Match.mean() + np.abs(boot_quant[1])).round(3)]
print("Boot Confidence Interval2:",boot_ci2)

# Plot histogram and KDE
ax = bm.samples.plot(kind='hist')
bm.samples.plot(kind='kde', ax=ax, secondary_y=True)
ax.vlines(boot_quant[0], ymin = 0, ymax = 3500,colors='red')
ax.vlines(boot_quant[1], ymin = 0, ymax = 3500,colors='purple')
plt.show()

### Example 2 CIs
yhat_s    = np.exp(model.fit(Xtrain, ytrain).predict(Xtest)) # sample statistic
mean_yhat_boot = BootstrapPred(model, Xtrain, ytrain, Xtest) # bootstrap statistic
### CI with bootsrap
# Bootstrap with CL of 99%
boot_CL = 99/100 

p_1 = (1-boot_CL)/2
p_2 = 1-p_1

boot_quant = np.quantile(mean_yhat_boot-yhat_s.mean(), [p_1, p_2])

boot_ci = [((yhat_s.mean() - boot_quant[1])*1e-6).round(3), 
           ((yhat_s.mean() - boot_quant[0])*1e-6).round(3)]
print("Boot confidence interval is",boot_ci,'- in million euros.')
bm = pd.DataFrame(data=BootstrapPred(model, Xtrain, ytrain, Xtest, 400)-yhat_s.mean(), columns=['Values'])
ax = bm.Values.plot(kind='hist', bins=20)
bm.Values.plot(kind='kde', ax=ax, secondary_y=True)
plt.show()

### CI with CLT
# Calculate CI using CLT:

stderr = yhat_s.std() / np.sqrt(yhat_s.shape[0])

cl = 0.99
print(ss.norm.ppf(cl))
lower = yhat_s.mean() - ss.norm.ppf(cl) * stderr
upper = yhat_s.mean() + ss.norm.ppf(cl) * stderr

print('The confidence interval is [%.3f, %.3f] - in million euros.' % (lower*1e-6, upper*1e-6))

### CI with t-distribution
n = 30
alpha = (1/100)
crit_val = 1-(alpha/2)

yhat_s_new = np.random.choice(yhat_s, size=n)
dof = n-1
t_value = t.ppf(crit_val, df=dof)
stderr = yhat_s_new.std() / np.sqrt(n)
lower = yhat_s_new.mean() - t_value * stderr 
upper = yhat_s_new.mean() + t_value * stderr

print('The confidence interval is [%.3f, %.3f] - in million euros.' % (lower*1e-6, upper*1e-6))
